In [ ]:
import ee
import geemap

ee.Initialize()

# Export image collection 

In [ ]:
point = ee.Geometry.Point(-99.2222, 46.7816)
collection = (
    ee.ImageCollection("USDA/NAIP/DOQQ")
    .filterBounds(point)
    .filterDate("2008-01-01", "2018-01-01")
    .filter(ee.Filter.listContains("system:band_names", "N"))
)

In [ ]:
collection.aggregate_array("system:index").getInfo()

In [ ]:
collection

In [ ]:
out_dir = "naip"
geemap.ee_export_image_collection(collection, out_dir=out_dir, scale=10)

### To drive

In [ ]:
geemap.ee_export_image_collection_to_drive(collection, folder="export", scale=10)

In [ ]:
geemap.image_dates(collection).sort()

In [ ]:
collection

In [ ]:
m = geemap.Map()
m.addLayer(collection.first(), {}, "First image")
m.centerObject(point, 12)
m

In [ ]:
m.user_roi

In [ ]:
ts = geemap.naip_timeseries(roi=m.user_roi, start_year=2009, end_year=2010, RGBN=True)

### Create a time series for satellite images
`geemap.create_timeseries()` or `geemap.naip_timeseries()`

In [ ]:
ts

In [ ]:
m = geemap.Map()
image = ts.first()
m.addLayer(image, {}, "First image")
m.center_object(ts, 12)
m

In [ ]:
roi = ee.Geometry.Point([-122.44, 37.75])
collection = (
    ee.ImageCollection("LANDSAT/LC08/C02/T1_TOA")
    .filterBounds(roi)
    .sort("CLOUD_COVER")
    .limit(10)
)

image = collection.first()

Map = geemap.Map()

vis_params = {
    "bands": ["B5", "B4", "B3"],
    "min": 0,
    "max": 0.3,
    "gamma": [0.95, 1.1, 1],
}

Map.addLayer(image, vis_params, "LANDSAT 8")
Map.setCenter(-122.44, 37.75, 8)
Map

## Exporting image thumbnails

In [ ]:
out_img = "landsat.jpg"
region = ee.Geometry.BBox(-122.5955, 37.5339, -122.0982, 37.8252)
geemap.get_image_thumbnail(image, out_img, vis_params, dimensions=1000, region=region)

In [ ]:
geemap.show_image(out_img)

### Downloading several thumbnails images at onece

`geemap.get_image_collection_thumbnails()`


Download thumbnails one by one

```python
geemap.get_image_collection_thumbnails(collection, out_dir, vis_params, dimensions=512, region=region)

```

In [ ]:
out_dir = "thumbnails"
geemap.get_image_collection_thumbnails(
    collection, out_dir, vis_params, dimensions=1000, region=region
)

### Make GIF file based on export thumbnails

In [ ]:
geemap.make_gif("thumbnails", "landsat.gif", fps=3)

# Download feature collections (vector data)

In [ ]:
Map = geemap.Map()
fc = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017").filter(
    ee.Filter.eq("country_co", "KV")
)

Map.addLayer(fc, {}, "Europe")
Map.centerObject(fc, 10)
Map

In [ ]:
geemap.ee_to_shp(fc, filename="kv.shp")

### exporting functions ```geemap.ee_to_vector(fc, filname='.shp,.geojson,.csv')```

In [ ]:
geemap.ee_export_vector(fc, filename="kv.geojson")

In [ ]:
gdf = geemap.ee_to_gdf(fc)
gdf

In [ ]:
df = geemap.ee_to_df(fc)
df

In [ ]:
gdf.explore()

# Exporting maps

In [ ]:
m = geemap.Map()
image = ee.Image("USGS/SRTMGL1_003")
vis_params = {
    "min": 0,
    "max": 5000,
    "palette": ["006633", "E5FFCC", "662A00", "D8D8D8", "F5F5F5"],
}
m.addLayer(image, vis_params, "SRTM DEM")
m

In [ ]:
m.to_html("dem.html", title="Earth Engine Map", width="100%", height="800px")
# downside: only works around 24 hours, the layer is temporary.
# Web App will works all the time